# Yusuf Demir 2210356074 
# Ender Orman 2210765011

# Resources and Trained Models

All trained models, and output results used in this notebook can be accessed via the following Google Drive link:

**https://drive.google.com/drive/folders/1LtKFVcK-MQ-cy0ft0OnkRs_rpbahDMID?usp=sharing**

Please make sure to mount the drive or download the necessary files before running the notebook.

In [ ]:
# -------------------------------------------
# Required Imports
# -------------------------------------------
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import pandas as pd
from datetime import datetime
import torchvision.transforms as T
from tqdm import tqdm

In [ ]:
# -------------------------------------------
# Custom Dataset Class for LOL Dataset
# -------------------------------------------
class LOLDataset(Dataset):
    def __init__(self, root_dir, split="our485", transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        
        # Define low and high light directories
        self.low_light_dir = os.path.join(root_dir, split, "low")
        self.high_light_dir = os.path.join(root_dir, split, "high")

        # Filter valid image extensions
        valid_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".tiff")
        self.image_names = [
            f for f in os.listdir(self.low_light_dir) if f.lower().endswith(valid_extensions)
        ]

    def __len__(self):
        # Return the number of image pairs
        return len(self.image_names)

    def __getitem__(self, idx):
        # Get image filenames for both low and high light versions
        img_name = self.image_names[idx]
        low_light_path = os.path.join(self.low_light_dir, img_name)
        high_light_path = os.path.join(self.high_light_dir, img_name)

        # Open both images and convert to RGB
        low_light_image = Image.open(low_light_path).convert("RGB")
        high_light_image = Image.open(high_light_path).convert("RGB")

        # Apply transformations if provided
        if self.transform:
            low_light_image = self.transform(low_light_image)
            high_light_image = self.transform(high_light_image)

        return low_light_image, high_light_image

# -------------------------------------------
# Image Transformations
# -------------------------------------------
# Resizing all images to the same size and converting them to tensors
transform = transforms.Compose([
    transforms.ToTensor()
])

# -------------------------------------------
# Load LOL Dataset
# -------------------------------------------
# Using 'our485' for training
dataset = LOLDataset(root_dir="/kaggle/input/loldataset", split="our485", transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# -------------------------------------------
# Visualization Function
# -------------------------------------------
def show_samples():
    # Load a batch of samples
    low_images, high_images = next(iter(dataloader))
    
    # Plot low-light (top row) and high-light (bottom row) image pairs
    fig, axes = plt.subplots(2, 4, figsize=(10, 5))
    for i in range(4):
        axes[0, i].imshow(low_images[i].permute(1, 2, 0))
        axes[0, i].set_title("Low Light")
        axes[0, i].axis("off")

        axes[1, i].imshow(high_images[i].permute(1, 2, 0))
        axes[1, i].set_title("High Light")
        axes[1, i].axis("off")
    
    plt.tight_layout()
    plt.show()

# -------------------------------------------
# Show Sample Images
# -------------------------------------------
show_samples()


In [ ]:
def compute_average_ssim_between_low_and_high(root_dir, resize_to=(256, 256)):
    low_dir = os.path.join(root_dir, "our485", "low")
    high_dir = os.path.join(root_dir, "our485", "high")

    filenames = sorted([
        f for f in os.listdir(low_dir)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ])

    total_ssim = 0
    count = 0

    transform = T.Compose([
        T.Resize(resize_to),
        T.ToTensor()
    ])

    for name in tqdm(filenames, desc="Calculating SSIM"):
        low_path = os.path.join(low_dir, name)
        high_path = os.path.join(high_dir, name)

        if not os.path.exists(high_path):
            continue

        low_img = transform(Image.open(low_path).convert("RGB")).numpy().transpose(1, 2, 0)
        high_img = transform(Image.open(high_path).convert("RGB")).numpy().transpose(1, 2, 0)

        win_size = min(low_img.shape[0], low_img.shape[1], 7)

        score = ssim(low_img, high_img, data_range=1, channel_axis=2, win_size=win_size)
        total_ssim += score
        count += 1

    avg_ssim = total_ssim / count if count > 0 else 0
    print(f"\nAverage SSIM between low-light and high-light images: {avg_ssim:.4f}")
    return avg_ssim

compute_average_ssim_between_low_and_high("/kaggle/input/loldataset")

In [ ]:
import torch
import torch.nn as nn

class SRCNN(nn.Module):
    def __init__(self, input_channels=3, output_channels=3):
        super(SRCNN, self).__init__()
        
        # First convolution layer: large kernel to extract features from low-resolution image
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=9, padding=4)
        
        # Second layer: map high-dimensional features to a lower-dimensional space
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=2)
        
        # Final layer: reconstruct the high-resolution image
        self.conv3 = nn.Conv2d(32, output_channels, kernel_size=5, padding=2)
        
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x + residual


In [ ]:
# Residual block for deeper learning
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        )

    def forward(self, x):
        return x + self.block(x)

# Improved SRCNN with upsampling and residual blocks
class EnhancedSRCNN(nn.Module):
    def __init__(self, input_channels=3, output_channels=3, upscale_factor=2):
        super(EnhancedSRCNN, self).__init__()

        self.entry = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=9, padding=4),
            nn.ReLU(inplace=True)
        )

        self.resblock1 = ResidualBlock(64)
        self.resblock2 = ResidualBlock(64)

        # Output layer
        self.exit = nn.Conv2d(64, output_channels, kernel_size=5, padding=2)

    def forward(self, x):
        x = self.entry(x)
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.exit(x)
        return x

In [ ]:
import torch
import torch.nn as nn

# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        )

    def forward(self, x):
        return x + self.block(x)

# Squeeze-and-Excitation Block (Channel Attention)
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(channels, channels // reduction, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        scale = self.pool(x)
        scale = self.fc(scale)
        return x * scale

# Enhanced SRCNN v2
class EnhancedSRCNNv2(nn.Module):
    def __init__(self, input_channels=3, output_channels=3):
        super(EnhancedSRCNNv2, self).__init__()

        self.entry = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=9, padding=4),
            nn.ReLU(inplace=True)
        )

        self.resblock1 = ResidualBlock(64)
        self.resblock2 = ResidualBlock(64)
        self.att1 = SEBlock(64)

        self.resblock3 = ResidualBlock(64)
        self.resblock4 = ResidualBlock(64)
        self.att2 = SEBlock(64)

        self.exit = nn.Conv2d(64, output_channels, kernel_size=5, padding=2)

    def forward(self, x):
        x = self.entry(x)
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.att1(x)
        x = self.resblock3(x)
        x = self.resblock4(x)
        x = self.att2(x)
        x = self.exit(x)
        return x

In [ ]:
class CBAMBlock(nn.Module):
    def __init__(self, channels, reduction_ratio=16):
        super(CBAMBlock, self).__init__()

        # Channel Attention
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.channel_attention = nn.Sequential(
            nn.Conv2d(channels, channels // reduction_ratio, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction_ratio, channels, 1, bias=False)
        )
        self.sigmoid_channel = nn.Sigmoid()

        # Spatial Attention
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        # ----- Channel Attention -----
        avg_out = self.channel_attention(self.avg_pool(x))
        max_out = self.channel_attention(self.max_pool(x))
        channel_attention = self.sigmoid_channel(avg_out + max_out)
        x = x * channel_attention

        # ----- Spatial Attention -----
        avg_channel = torch.mean(x, dim=1, keepdim=True)
        max_channel, _ = torch.max(x, dim=1, keepdim=True)
        spatial_input = torch.cat([avg_channel, max_channel], dim=1)
        spatial_attention = self.spatial_attention(spatial_input)
        x = x * spatial_attention

        return x

class YASRNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(YASRNet, self).__init__()

        self.entry = nn.Sequential(
            nn.Conv2d(in_channels, 64, 9, padding=4),
            nn.ReLU()
        )

        self.block1 = ResidualBlock(64)
        self.block2 = ResidualBlock(64)
        self.cbam1 = CBAMBlock(64)

        self.block3 = ResidualBlock(64)
        self.block4 = ResidualBlock(64)
        self.cbam2 = CBAMBlock(64)

        self.upsample = nn.Sequential(
            nn.Conv2d(64, 256, kernel_size=3, padding=1),
            nn.PixelShuffle(2),
            nn.ReLU()
        )

        self.out = nn.Conv2d(64, out_channels, 5, padding=2)

    def forward(self, x):
        x = self.entry(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.cbam1(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.cbam2(x)
        x = self.upsample(x)
        return self.out(x)

In [ ]:
# --------------------------
# Model Training Function
# --------------------------

def train_model(model, dataloader, num_epochs=10, lr=0.001, model_name="SRCNN"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_loss = float("inf")
    best_model_wts = None
    loss_history = []

    # Create output dirs
    os.makedirs("saved_models", exist_ok=True)
    os.makedirs("Metrics", exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    metrics_dir = os.path.join("Metrics", f"{model_name}_{timestamp}")
    os.makedirs(metrics_dir, exist_ok=True)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        for low_light, high_light in dataloader:
            low_light, high_light = low_light.to(device), high_light.to(device)

            optimizer.zero_grad()
            outputs = model(low_light)

            if outputs.size() != high_light.size():
                high_light = F.interpolate(high_light, size=outputs.shape[-2:], mode="bilinear", align_corners=False)

            loss = criterion(outputs, high_light)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        loss_history.append(avg_loss)

        print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {avg_loss:.4f}")

        if avg_loss < best_loss:
            best_loss = avg_loss
            best_model_wts = model.state_dict()
            best_epoch = epoch + 1

    # Save best model
    if best_model_wts:
        filename = f"{model_name}_{num_epochs}e_best.pth"
        save_path = os.path.join("saved_models", filename)
        torch.save(best_model_wts, save_path)
        model.load_state_dict(best_model_wts)

    # Save loss plot
    plt.figure(figsize=(8, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o', color='blue')
    plt.title(f"{model_name} Training Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss (MSE)")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(metrics_dir, f"{model_name}_loss.png"))
    plt.close()

    print(f"Loss plot saved to '{metrics_dir}/'.")

    return model


In [ ]:
# --------------------------
# Visualization Function (Low-Memory Safe)
# --------------------------

def test_model(model, low_light, high_light, max_visualize=4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    # Restrict batch size to first N samples
    low_light = low_light[:max_visualize].to(device)
    high_light = high_light[:max_visualize].to("cpu")

    with torch.no_grad():
        outputs = model(low_light).cpu()

    # Plot the selected samples
    fig, axes = plt.subplots(3, max_visualize, figsize=(4 * max_visualize, 8))

    for i in range(max_visualize):
        axes[0, i].imshow(low_light[i].cpu().permute(1, 2, 0).clamp(0, 1))
        axes[0, i].set_title("Low Light")
        axes[0, i].axis("off")

        axes[1, i].imshow(outputs[i].permute(1, 2, 0).clamp(0, 1))
        axes[1, i].set_title("Super Resolution")
        axes[1, i].axis("off")

        axes[2, i].imshow(high_light[i].permute(1, 2, 0).clamp(0, 1))
        axes[2, i].set_title("Ground Truth")
        axes[2, i].axis("off")

    plt.tight_layout()
    plt.show()



In [ ]:
# --------------------------
# Evaluation Metrics: PSNR & SSIM
# --------------------------

# We will keep all trials here globally
performance_log = []

def evaluate_performance(model, dataloader, model_name="SRCNN", epoch_count=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    psnr_scores = []
    ssim_scores = []

    with torch.no_grad():
        for low_light, high_light in dataloader:
            low_light, high_light = low_light.to(device), high_light.to(device)
            outputs = model(low_light)

            for i in range(outputs.size(0)):
                output = outputs[i]
                target = high_light[i]

                # Match size if needed
                if output.shape[-2:] != target.shape[-2:]:
                    target = F.interpolate(target.unsqueeze(0), size=output.shape[-2:], mode="bilinear", align_corners=False).squeeze(0)

                # Convert to numpy
                pred = output.cpu().permute(1, 2, 0).numpy().clip(0, 1)
                gt = target.cpu().permute(1, 2, 0).numpy().clip(0, 1)

                h, w, _ = gt.shape
                win_size = min(h, w, 7)

                psnr_scores.append(psnr(gt, pred, data_range=1))
                ssim_scores.append(ssim(gt, pred, data_range=1, win_size=win_size, channel_axis=2))

    mean_psnr = np.mean(psnr_scores)
    mean_ssim = np.mean(ssim_scores)

    print(f"\nArchitecture: {model_name}, Epochs: {epoch_count}")
    print(f"Average PSNR: {mean_psnr:.2f}")
    print(f"Average SSIM: {mean_ssim:.4f}")

    # Save to global log
    performance_log.append({
        "Model": model_name,
        "Epochs": epoch_count,
        "PSNR": round(mean_psnr, 2),
        "SSIM": round(mean_ssim, 4)
    })

In [ ]:
def compare_all_models():
    if not performance_log:
        print("No evaluations logged yet.")
        return

    df = pd.DataFrame(performance_log)
    df_sorted = df.sort_values(by="SSIM", ascending=False)

    print("\n Model Performance Comparison:")
    print(df_sorted.to_string(index=False))

    # Create bar chart for SSIM and PSNR
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    # SSIM plot
    ax[0].bar(df_sorted["Model"] + " (" + df_sorted["Epochs"].astype(str) + "e)", df_sorted["SSIM"], color="skyblue")
    ax[0].set_title("SSIM Comparison")
    ax[0].set_ylabel("SSIM Score")
    ax[0].set_ylim(0, 1)
    ax[0].tick_params(axis='x', rotation=45)

    # PSNR plot
    ax[1].bar(df_sorted["Model"] + " (" + df_sorted["Epochs"].astype(str) + "e)", df_sorted["PSNR"], color="salmon")
    ax[1].set_title("PSNR Comparison")
    ax[1].set_ylabel("PSNR (dB)")
    ax[1].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

In [ ]:
import time

# --------- Global Configurations ----------
NUM_EPOCH = 30
LR = 0.001
BATCH_SIZE = 32

performance_log = []
training_times = {}
trained_models = {}

In [ ]:
print("\nTraining SRCNN...")

start_time = time.time()
model_srcnn = SRCNN()
trained_srcnn = train_model(
    model_srcnn,
    dataloader,
    num_epochs=NUM_EPOCH,
    lr=LR,
    model_name="SRCNN"
)
training_times["SRCNN"] = time.time() - start_time
trained_models["SRCNN"] = trained_srcnn

# Fixed test batch
fixed_batch = next(iter(DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)))
low_light_batch, high_light_batch = fixed_batch

print("\nVisual Comparison: SRCNN")
test_model(trained_srcnn, low_light_batch, high_light_batch)

evaluate_performance(trained_srcnn, dataloader, model_name="SRCNN", epoch_count=NUM_EPOCH)

In [ ]:
print("\nTraining EnhancedSRCNN...")

start_time = time.time()
model_enhanced = EnhancedSRCNN()
trained_enhanced = train_model(
    model_enhanced,
    dataloader,
    num_epochs=NUM_EPOCH,
    lr=LR,
    model_name="EnhancedSRCNN"
)
training_times["EnhancedSRCNN"] = time.time() - start_time
trained_models["EnhancedSRCNN"] = trained_enhanced

# Aynı test batch'i kullan
print("\nVisual Comparison: EnhancedSRCNN")
test_model(trained_enhanced, low_light_batch, high_light_batch)

evaluate_performance(trained_enhanced, dataloader, model_name="EnhancedSRCNN", epoch_count=NUM_EPOCH)

In [ ]:
print("\nTraining EnhancedSRCNNv2...")
start_time = time.time()
model_v2 = EnhancedSRCNNv2()
trained_v2 = train_model(model_v2, dataloader, NUM_EPOCH, LR, "EnhancedSRCNNv2")
training_times["EnhancedSRCNNv2"] = time.time() - start_time
trained_models["EnhancedSRCNNv2"] = trained_v2

print("\nVisual Comparison: EnhancedSRCNNv2")
test_model(trained_v2, low_light_batch, high_light_batch)
evaluate_performance(trained_v2, dataloader, model_name="EnhancedSRCNNv2", epoch_count=NUM_EPOCH)

In [ ]:
print("\nTraining YASRNet...")
start_time = time.time()

model_yasr = YASRNet()
trained_yasr = train_model(
    model_yasr,
    dataloader,
    num_epochs=NUM_EPOCH,
    lr=LR,
    model_name="YASRNet"
)

training_times["YASRNet"] = time.time() - start_time
trained_models["YASRNet"] = trained_yasr

In [ ]:
# Test visualization (comparison with the same batch)
print("\nVisual Comparison: YASRNet")
test_model(trained_yasr, low_light_batch, high_light_batch, max_visualize=4)

# Performance evaluation (PSNR + SSIM)
evaluate_performance(trained_yasr, dataloader, model_name="YASRNet", epoch_count=NUM_EPOCH) 

Duruma göre alttaki silinecek

In [ ]:
# --------------------------------------------
# Example: How to Load a Pretrained Model
# --------------------------------------------

# # 1. Make sure the model class is defined
# model = YASRNet()

# # 2. Load the saved best weights
# model.load_state_dict(torch.load("/kaggle/working/saved_models/YASRNet_30e_best.pth",
#                                  map_location="cuda" if torch.cuda.is_available() else "cpu"))

# # 3. Set the model to evaluation mode and move to device
# model = model.eval().to("cuda" if torch.cuda.is_available() else "cpu")

# # 4. Register the model to dictionary (optional)
# trained_models["YASRNet"] = model

# # 5. Evaluate model performance (e.g., PSNR + SSIM)
# evaluate_performance(model, dataloader, model_name="YASRNet", epoch_count=NUM_EPOCH)


# Edge-Aware Loss for Better Detection

In this experiment, we update the loss function by combining the traditional MSE loss with an edge-based loss using the Laplacian filter. The goal is to preserve sharp structures and edges in the super-resolved images.

Although this may lead to slightly lower SSIM scores (since the overall pixel similarity may decrease), we expect object detection performance to improve. This is because detectors like YOLO rely more on edges and object boundaries rather than smooth textures.

**Total Loss = 0.8 × MSE + 0.2 × Edge Loss (Laplacian)**

This loss encourages the model to generate outputs that are not only close to the ground truth but also maintain important edge details for better semantic understanding.


In [ ]:
def get_laplacian_kernel(device):
    lap = torch.tensor([[0, 1, 0],
                        [1, -4, 1],
                        [0, 1, 0]], dtype=torch.float32)
    kernel = lap.unsqueeze(0).unsqueeze(0)  # (1, 1, 3, 3)
    return kernel.to(device)


def train_model_edge_mse(model, dataloader, num_epochs=10, lr=0.001, model_name="SR_EdgeMSE"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    mse_loss_fn = nn.MSELoss()
    laplacian_kernel = get_laplacian_kernel(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    best_loss = float("inf")
    best_model_wts = None
    loss_history = []

    # Output dirs
    os.makedirs("saved_models", exist_ok=True)
    os.makedirs("Metrics", exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    metrics_dir = os.path.join("Metrics", f"{model_name}_{timestamp}")
    os.makedirs(metrics_dir, exist_ok=True)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        for low_light, high_light in dataloader:
            low_light, high_light = low_light.to(device), high_light.to(device)
            optimizer.zero_grad()

            outputs = model(low_light)

            if outputs.shape != high_light.shape:
                high_light = F.interpolate(high_light, size=outputs.shape[-2:], mode="bilinear", align_corners=False)

            # MSE Loss
            mse = mse_loss_fn(outputs, high_light)

            # Edge Loss (Laplacian)
            edge_sr = F.conv2d(outputs, laplacian_kernel.expand(outputs.size(1), -1, -1, -1),
                               padding=1, groups=outputs.size(1))
            edge_gt = F.conv2d(high_light, laplacian_kernel.expand(high_light.size(1), -1, -1, -1),
                               padding=1, groups=high_light.size(1))
            edge_loss = mse_loss_fn(edge_sr, edge_gt)

            # Total Loss
            total = 0.8 * mse + 0.2 * edge_loss
            total.backward()
            optimizer.step()

            total_loss += total.item()

        avg_loss = total_loss / len(dataloader)
        loss_history.append(avg_loss)

        print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {avg_loss:.4f}")

        if avg_loss < best_loss:
            best_loss = avg_loss
            best_model_wts = model.state_dict()
            best_epoch = epoch + 1

    # Save best model
    if best_model_wts:
        save_path = os.path.join("saved_models", f"{model_name}_{num_epochs}e_best.pth")
        torch.save(best_model_wts, save_path)
        model.load_state_dict(best_model_wts)

    # Plot loss
    plt.figure(figsize=(8, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o', color='purple')
    plt.title(f"{model_name} Training Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss (MSE + Edge)")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(metrics_dir, f"{model_name}_loss.png"))
    plt.close()

    print(f"Metrics saved to '{metrics_dir}/'")
    return model


In [ ]:
print("\nTraining SRCNN (MSE + Edge Loss)...")

start_time = time.time()

model_srcnn = SRCNN()
trained_srcnn = train_model_edge_mse(
    model_srcnn,
    dataloader,
    num_epochs=NUM_EPOCH,
    lr=LR,
    model_name="SRCNN_Edge"
)

training_times["SRCNN_Edge"] = time.time() - start_time
trained_models["SRCNN_Edge"] = trained_srcnn

# Fixed test batch (değişmedi)
fixed_batch = next(iter(DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)))
low_light_batch, high_light_batch = fixed_batch

print("\nVisual Comparison: SRCNN_Edge")
test_model(trained_srcnn, low_light_batch, high_light_batch)

evaluate_performance(trained_srcnn, dataloader, model_name="SRCNN_Edge", epoch_count=NUM_EPOCH)

In [ ]:
# 1. Trained model names
print("Trained Models:")
print(list(trained_models.keys()))

# 2. Training periods
print("\nTraining Time Summary:")
for model_name, duration in training_times.items():
    print(f"{model_name}: {duration:.2f} seconds")


In [ ]:
compare_all_models()

# Choose the best model by SSIM
best_model_name = max(performance_log, key=lambda x: x["SSIM"])["Model"]
print(f"\nUsing best SR model based on SSIM: {best_model_name}")
best_model = trained_models[best_model_name]

### SSIM Evaluation Result

Among all tested models, **YASRNet achieved the highest SSIM score (0.8367)**.  
This is likely due to its deeper structure and attention modules (CBAM), which help preserve fine details and textures in the image.  
Other models, including edge-aware SRCNN, focus more on structural sharpness but may sacrifice overall similarity.


In [ ]:
# Create SRresults folder and save results
import torchvision.transforms as T
import shutil

output_dir = "SRresults"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print("old folder deleted")
os.makedirs(output_dir, exist_ok=True)

ordered_dataset = LOLDataset(root_dir="/kaggle/input/loldataset", split="our485", transform=transform)
ordered_loader = DataLoader(ordered_dataset, batch_size=4, shuffle=False)
image_names = ordered_dataset.image_names

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model.to(device)
best_model.eval()

with torch.no_grad():
    for idx, (low_img, _) in enumerate(ordered_loader):
        low_img = low_img.to(device)
        sr_output = best_model(low_img)

        for i in range(sr_output.size(0)):
            img_tensor = sr_output[i].cpu().clamp(0, 1)
            img_pil = T.ToPILImage()(img_tensor)
            filename = image_names[idx * ordered_loader.batch_size + i]
            img_pil.save(os.path.join(output_dir, filename))

print(f"\nSR images saved to '{output_dir}/'")


In [ ]:
best_model = trained_models["SRCNN_Edge"]

In [ ]:
# Create SRresults folder and save results
import torchvision.transforms as T
import shutil

output_dir = "SRresults_edge"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print("old folder deleted")
os.makedirs(output_dir, exist_ok=True)

ordered_dataset = LOLDataset(root_dir="/kaggle/input/loldataset", split="our485", transform=transform)
ordered_loader = DataLoader(ordered_dataset, batch_size=4, shuffle=False)
image_names = ordered_dataset.image_names

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model.to(device)
best_model.eval()

with torch.no_grad():
    for idx, (low_img, _) in enumerate(ordered_loader):
        low_img = low_img.to(device)
        sr_output = best_model(low_img)

        for i in range(sr_output.size(0)):
            img_tensor = sr_output[i].cpu().clamp(0, 1)
            img_pil = T.ToPILImage()(img_tensor)
            filename = image_names[idx * ordered_loader.batch_size + i]
            img_pil.save(os.path.join(output_dir, filename))

print(f"\nSR images saved to '{output_dir}/'")


## YOLOv8l for Object Detection Evaluation

We use the pre-trained **YOLOv8l** (large) model to evaluate object detection performance on different versions of the same image:
- Low-light image
- Super-resolved image (enhanced from low-light)
- Ground-truth high-light image

YOLOv8l is selected because:
- It offers strong object detection accuracy out-of-the-box.
- It works well in complex indoor scenes without requiring fine-tuning.
- It balances performance and speed effectively on GPU.

In this project, we do **not fine-tune** YOLO. Instead, we directly apply the pre-trained model to all image types and compare the number of detected objects and confidence scores.


In [ ]:
from ultralytics import YOLO

# Load YOLOv8m pre-trained model
model = YOLO("yolov8l.pt")  

# Inference on an image or tensor
results = model("/kaggle/input/loldataset/our485/high/103.png")  # Replace with your image path
results[0].show()

In [ ]:
# --------------------------
# Detection Comparison Function (Flexible SR folder)
# --------------------------

def compare_detections(model, image_id, sr_dir="/kaggle/working/SRresults"):
    """
    Performs YOLOv8 detection on high-light, low-light, and SR versions of the same image.
    Shows bounding boxes for visual comparison.

    Parameters:
        model: Pretrained YOLOv8 model
        image_id: Image file name, e.g., "5.png"
        sr_dir: Path to super-resolved image folder (default: '/kaggle/working/SRresults')
    """

    # Define full paths
    high_path = f"/kaggle/input/loldataset/our485/high/{image_id}"
    low_path = f"/kaggle/input/loldataset/our485/low/{image_id}"
    sr_path = os.path.join(sr_dir, image_id)

    # Perform inference
    print(f"\nHigh-light image: {image_id}")
    high_result = model(high_path)[0]
    high_result.show()

    print(f"\nLow-light image: {image_id}")
    low_result = model(low_path)[0]
    low_result.show()

    print(f"\nSuper-resolved image: {image_id}")
    sr_result = model(sr_path)[0]
    sr_result.show()

compare_detections(model, "5.png")
compare_detections(model, "5.png", sr_dir="/kaggle/working/SRresults_edge")

In [ ]:
def evaluate_detection_performance(
    model,
    sr_dir="/kaggle/working/SRresults",
    csv_path="detection_results.csv",
    confidence_threshold=0.8
):
    """
    Runs YOLO detection on low-light, high-light, and SR images and saves results to CSV.
    Also prints total detection counts for each image type.

    Parameters:
        model: Pretrained YOLO model
        sr_dir: Folder containing super-resolved images (default: '/kaggle/working/SRresults')
        csv_path: Output CSV file name (default: 'detection_results.csv')
        confidence_threshold: Minimum confidence to count a detection (default: 0.8)
    """
    import os
    import torch
    import pandas as pd
    from datetime import datetime

    low_dir = "/kaggle/input/loldataset/our485/low"
    high_dir = "/kaggle/input/loldataset/our485/high"

    # Backup existing CSV
    if os.path.exists(csv_path):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M")
        new_name = f"{csv_path.split('.')[0]}_{timestamp}.csv"
        os.rename(csv_path, new_name)
        print(f"Previous '{csv_path}' found. Renamed to '{new_name}'.")

    results = []
    image_names = sorted(os.listdir(low_dir))

    total_low, total_sr, total_high = 0, 0, 0

    for name in image_names:
        row = {"image": name}

        # Low-light image
        low_path = os.path.join(low_dir, name)
        low_result = model(low_path)[0]
        low_conf = low_result.boxes.conf.cpu() if len(low_result.boxes) > 0 else torch.tensor([])
        low_mask = low_conf >= confidence_threshold
        low_count = int(low_mask.sum())
        total_low += low_count
        row["low_detections"] = low_count
        row["low_conf_avg"] = float(low_conf[low_mask].mean().item()) if low_count > 0 else 0

        # Super-resolved image
        sr_path = os.path.join(sr_dir, name)
        if os.path.exists(sr_path):
            sr_result = model(sr_path)[0]
            sr_conf = sr_result.boxes.conf.cpu() if len(sr_result.boxes) > 0 else torch.tensor([])
            sr_mask = sr_conf >= confidence_threshold
            sr_count = int(sr_mask.sum())
            total_sr += sr_count
            row["sr_detections"] = sr_count
            row["sr_conf_avg"] = float(sr_conf[sr_mask].mean().item()) if sr_count > 0 else 0
        else:
            row["sr_detections"] = None
            row["sr_conf_avg"] = None

        # High-light image
        high_path = os.path.join(high_dir, name)
        high_result = model(high_path)[0]
        high_conf = high_result.boxes.conf.cpu() if len(high_result.boxes) > 0 else torch.tensor([])
        high_mask = high_conf >= confidence_threshold
        high_count = int(high_mask.sum())
        total_high += high_count
        row["high_detections"] = high_count
        row["high_conf_avg"] = float(high_conf[high_mask].mean().item()) if high_count > 0 else 0

        results.append(row)

    # Save results to CSV
    df = pd.DataFrame(results)
    df.to_csv(csv_path, index=False)
    print(f"\nResults saved to {csv_path} (conf ≥ {confidence_threshold})")

    # Print summary
    print("\nTotal Detections Summary:")
    print(f"- Low-light images:      {total_low} detections")
    print(f"- Super-resolved images: {total_sr} detections")
    print(f"- High-light images:     {total_high} detections")


# Varsayılan SR klasörüyle:
evaluate_detection_performance(model)

In [ ]:
# Edge-enhanced SR sonucu için:
evaluate_detection_performance(model, sr_dir="/kaggle/working/SRresults_edge", csv_path="results_edge.csv")

In [ ]:
# --------------------------
# Detection Results Plotting Function
# --------------------------

def plot_detection_totals(csv_path="detection_results.csv"):
    """
    Reads a detection results CSV and plots total number of detections
    for low-light, super-resolved, and high-light images.

    Parameters:
        csv_path: Path to the detection results CSV file
    """
    import pandas as pd
    import matplotlib.pyplot as plt

    # Load CSV
    df = pd.read_csv(csv_path)

    # Replace NaN values with 0 for consistency
    df.fillna(0, inplace=True)

    # Calculate total detections
    totals = {
        "Low-light": int(df["low_detections"].sum()),
        "Super-resolved": int(df["sr_detections"].sum()),
        "High-light (GT)": int(df["high_detections"].sum())
    }

    # Plot
    plt.figure(figsize=(8, 6))
    plt.bar(totals.keys(), totals.values(), color=["gray", "skyblue", "lightgreen"])
    plt.title("Total Objects Detected by Image Type")
    plt.ylabel("Total Number of Detections")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

plot_detection_totals("detection_results.csv")
plot_detection_totals("results_edge.csv")

In [ ]:
compare_detections(model, "702.png")
compare_detections(model, "702.png", sr_dir="/kaggle/working/SRresults_edge")

In [ ]:
compare_detections(model, "5.png")
compare_detections(model, "5.png", sr_dir="/kaggle/working/SRresults_edge")

In [ ]:
compare_detections(model, "246.png")
compare_detections(model, "246.png", sr_dir="/kaggle/working/SRresults_edge")

In [ ]:
compare_detections(model, "242.png")
compare_detections(model, "242.png", sr_dir="/kaggle/working/SRresults_edge")

In [ ]:
compare_detections(model, "240.png")
compare_detections(model, "240.png", sr_dir="/kaggle/working/SRresults_edge")

In [ ]:
compare_detections(model, "645.png")
compare_detections(model, "645.png", sr_dir="/kaggle/working/SRresults_edge")

In [ ]:
compare_detections(model, "781.png")
compare_detections(model, "781.png", sr_dir="/kaggle/working/SRresults_edge")

In [ ]:
!zip -r Last.zip /kaggle/working